In [1]:
import pandas as pd

In [2]:
tariff_broadband_df = pd.read_excel(r'Тарифы.xlsx', names=['account', 'connected', 'tariff', 'IP', 'lease'])
payment_broadband_df = pd.read_excel(r'Оплата.xlsx', names=['account', 'datetime', 'amount', 'method']);
CC_call_df = pd.read_excel(r'Обращения в КЦ.xlsx', names=['date', 'time', 'duration', 'account', 'region', 'theme', 'type'])
change_tariff_broadband_df = pd.read_excel(r'Изменения тарифов.xlsx', names=['account', 'datetime', 'new_tariff', 'old_tariff']);

#tariff_broadband_df['connected'] = tariff_broadband_df['connected'].astype('datetime64').dt.date
#payment_broadband_df['datetime'] = payment_broadband_df['datetime'].astype('datetime64').dt.date
#CC_call_df['date'] = CC_call_df['date'].astype('datetime64').dt.date
change_tariff_broadband_df['datetime'] = change_tariff_broadband_df['datetime'].astype('datetime64').dt.date

<h4>1. Укажите три самых частых тарифа</h4>

In [3]:
print(tariff_broadband_df['tariff'].value_counts()[:3])

План А          10167
Средний план     2397
Камбэк 250       2396
Name: tariff, dtype: int64


<h4>2. Укажите месяц с самым большим количеством подключений</h4>

In [4]:
print("Месяц с максимальным кол. - ", tariff_broadband_df.set_index('connected').resample('M').size().idxmax().month)

Месяц с максимальным кол. -  10


<h4>3. Найдите корреляцию между наличием у абонента статического IP адреса и наличием роутера в аренде. Что можно сказать о связи друг с другом этих свойств?</h4>

In [5]:
print(tariff_broadband_df['IP'].corr(tariff_broadband_df['lease']))

-0.040385354255088504


-0.040385354255088504
Связь между этими свойствами очень слабая и она не достоверна. Она меньше критического значения корреляции Пирсона для степеней свободы больше 1000

<h4>4. Определите среднее, моду, медиану, 1 и 3 квартили (Q1 и Q3) поступивших платежей</h4>

In [6]:
print("Средний платеж - ",payment_broadband_df['amount'].mean())
print("Медиана платежей - ",payment_broadband_df['amount'].median())
print("Мода платежей - ",payment_broadband_df['amount'].mode())
print("Q1 - ",payment_broadband_df['amount'].quantile(q=0.25))
print("Q3 - ",payment_broadband_df['amount'].quantile(q=0.75))

Средний платеж -  481.58725996452915
Медиана платежей -  500.0
Мода платежей -  0    500.0
dtype: float64
Q1 -  250.0
Q3 -  590.0


<h4>5. Определите месяц с наибольшей суммой платежей</h4>

In [7]:
print("Месяц - ",payment_broadband_df.set_index('datetime').resample('M')['amount'].sum().idxmax().month)

Месяц -  12


<h4>6. Найдите 3 договора, заплативших наибольшую сумму</h4>

In [8]:
print(payment_broadband_df.groupby('account')['amount'].sum().sort_values()[-3:])

account
1219811    19516.0
1532729    20000.0
781442     29000.0
Name: amount, dtype: float64


<h4>7. Найдите самый частый метод оплаты у клиентов, подключившихся в октябре</h4>

In [9]:
start_date='2018-10-01'
end_date='2018-10-31'
mask = (payment_broadband_df['datetime'] >= start_date) & (payment_broadband_df['datetime'] <= end_date)
data_oct=payment_broadband_df.loc[mask]
print("Самый частый способ - ", data_oct['method'].value_counts().idxmax())

Самый частый способ -  Альфа


<h4>8. Найдите 3 договора, на которых наиболее часто меняли тарифы</h4>

In [10]:
print(change_tariff_broadband_df.groupby('account').size().sort_values()[-3:])

account
298677    20
157175    22
105594    26
dtype: int64


<h4>9. Укажите тариф, с которого "уходили" (переключались на другой) чащей всего</h4>

In [11]:
print(change_tariff_broadband_df.groupby('old_tariff').size().idxmax())

План А


<h4>10. Укажите договор клиента, который подключившись в 2018 году, обращался в КЦ чаще всего</h4>

In [12]:
klient=tariff_broadband_df.loc[tariff_broadband_df['connected'] >= '2018-01-01']
res_mas=CC_call_df.merge(klient)
print("Договор 2 - ",res_mas.groupby('account').size().idxmax())

Договор 2 -  244516


<h4>11. Укажите шестичасовой интервал, в котором было больше всего обращений в КЦ</h4>

In [13]:
CC_call_df['full_t']=pd.to_datetime(CC_call_df['date'].astype(str) +' '+ CC_call_df['time']).astype('datetime64')
start_date=CC_call_df['full_t'].min()
end_period=CC_call_df['full_t'].max()
start_date=start_date.normalize()
hours6=pd.Timedelta('6 hours')
hours1=pd.Timedelta('1 hours')
size_max=0
start_max=start_date
end_max=start_date
while start_date<end_period:
   start_date+=hours1
   end_date = start_date + hours6
   mask = (CC_call_df['full_t'] >= start_date) & (CC_call_df['full_t'] <= end_date)
   size_nabor=len(CC_call_df.loc[mask])
   if size_nabor>size_max:
      start_max = start_date
      end_max = end_date
      size_max=size_nabor
print('Max количество обращений - {0} с {1} по {2} '.format(size_max,start_max,end_max))

Max количество обращений - 469 с 2018-06-23 14:00:00 по 2018-06-23 20:00:00 


<h4>12. Определите среднее время разговора с КЦ клиентов из г. Обнинск</h4>

In [14]:
print("Среднее время - ",CC_call_df.loc[CC_call_df['region'] == 'г. Обнинск']['duration'].mean())

Среднее время -  232.44


<h4>13. Укажите 5 наиболее часто встречающихся типов обращений в КЦ (в порядке убывания их кол-ва) клиентов, которые подключились в 2018 году и сделали платежей на сумму от 10 000 руб.</h4>

In [15]:
klient=tariff_broadband_df.loc[tariff_broadband_df['connected'] >= '2018-01-01']
klient_oplata=payment_broadband_df.merge(klient, how='inner') 
klient_oplataSum=klient_oplata.groupby('account')['amount'].sum()
klient_occ=[[kkk,lll] for kkk,lll in zip(klient_oplataSum[klient_oplataSum.values>=10000].index, klient_oplataSum[klient_oplataSum.values>=10000])]
result=CC_call_df.merge(pd.DataFrame(klient_occ, columns=['account','summm']))
print(result.groupby('theme').size().sort_values(ascending=False)[:5])

theme
Тех. обращение          158
Не тех. обращение        53
Проблема Клиентские      15
Проблема Финансы          3
Проблема Телевидение      2
dtype: int64
